In [13]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
import matplotlib.pyplot as plt
import hvplot as hvp
import datetime
import altair as alt
%matplotlib inline

In [2]:
# Creating a range of dates
start_date = datetime.datetime(2018, 1, 1)
end_date = datetime.datetime(2023, 12, 31)
date_range = pd.date_range(start_date, end_date, freq='D')
# Dates df
dates_df = pd.DataFrame(date_range, columns=['Date'])
#print(dates_df.head(1825))

# Creating a range of random "prices"
portfolio_df = pd.DataFrame(
    {
    "fake1": np.random.uniform(1000, -500, 100),
    "fake2": np.random.uniform(1000, -500, 100),
    "fake3": np.random.uniform(1000, -500, 100)
    }
    )
f1_portfolio_df = portfolio_df["fake1"].to_frame()

# Concatinating and finalizing test data
returns = pd.concat([dates_df, portfolio_df], join = "inner", axis = 1)
returns.set_index('Date', inplace = True)
returns = pd.DataFrame(returns)
returns.head()

In [3]:
initial_investment = 10000

In [4]:
def cumulative_returns(df):
    return (1 + df).cumprod()

In [5]:
user_cr = cumulative_returns(portfolio_df)
user_cr

,fake1,fake2,fake3
0,-479.82,-493.03,351.46
1,57836.76,-134064.92,176571.66
2,28650119.88,57699502.32,131842850.53
3,3617819588.11,57218554742.80,-48305185651.85
4,-257528051240.47,-27244507281508.60,-27740192623145.52
...,...,...,...
95,-7972071981440524776547539071537770031039396053...,-4743105153601428239207943572442221754097997037...,-3012469811436924757166351733881725817985390397...
96,-6035694403311019687938991770667577575126963207...,-2912643707175353668163003769670778753834338446...,11268391418462026261230672997052966158129016405...
97,11332110595457437722363591512688828514783400198...,-2413123043331214749784477421943330954067332986...,71443476518462852478681769246839488639949623178...
98,-3161384451529218882238401070351922761500642363...,-2337129121272126681852557161865977775103106180...,16729057682066967710303856286047031312443298882...


In [6]:
def covariance(df, ticker, market):
    return df[ticker].cov(df[market])

In [7]:
user_covariance = covariance(portfolio_df, 'fake2', 'fake3')
user_covariance

18624.033133256282

In [8]:
def variance(df, market):
    return df[market].var()

In [9]:
user_variance = variance(portfolio_df, 'fake3')
user_variance

191593.09870678137

### Drawdown

In [10]:
def drawdown(df, tickers):
    Roll_Max = df[tickers].cummax()
    Daily_Drawdown = df[tickers]/Roll_Max - 1.0
    Max_Daily_Drawdown = Daily_Drawdown.cummin() * 100

In [11]:
user_drawdown = drawdown(portfolio_df, 'fake1')
user_drawdown

### Drawdown Chart

In [16]:
returns.reset_index()
chart = alt.Chart(returns.reset_index()).mark_bar().encode(
    x='Date:T',  # T specifies temporal data type for x-axis
    y='fake1:Q',  # Q specifies a quantitative data type for y-axis
    color=alt.condition(
        alt.datum.fake1 > 0,
        alt.value("Green"),  # The positive color
        alt.value("red")
), tooltip= ['Date:T', 'fake1:Q']
).properties(width=800)
chart = chart.interactive()
chart

SyntaxError: invalid syntax (1764296038.py, line 6)

In [24]:
#both are the same
getattr(alt.datum,'fake1') 
alt.datum.fake1

datum.fake1

In [28]:
def drawdown_chart(df, date, quant):
    '''
    return: is a dataframe
    date: str: should be a date column name
    quant: str: should be a quantitative data type
    '''
    chart = alt.Chart(df.reset_index()).mark_bar().encode(
       x = date +':T',  # T specifies temporal data type for x-axis
        y = quant +':Q',  # Q specifies a quantitative data type for y-axis
        color=alt.condition(
            getattr(alt.datum, quant) > 0,
            alt.value("Green"),  # The positive color
            alt.value("red")
    ), tooltip= [date +':T', quant +':Q']
    ).properties(width=800)
    chart = chart.interactive()
    return chart

In [29]:
drawdown_chart(returns, "Date", "fake1")

alt.Chart(...)

In [ ]:
def tracking_error(df, tickers, market):
    track_error = np.sqrt(sum([i**2 for i in df[tickers] - df[market]]))
    return track_error

In [ ]:
user_tracking_error = tracking_error(portfolio_df, 'fake2', 'fake3')
user_tracking_error

6588.122249293263

In [ ]:
def beta(covariance, variance):
    chart_beta = covariance / variance
    chart_beta = chart_beta.rolling(window = 21).plot()
    plt.show()
    beta.shape
    chart_beta.plot()
    return chart_beta

In [ ]:
user_beta = beta(user_covariance, user_variance)
user_beta


NameError: name 'chart_beta' is not defined

In [ ]:
def sharpe_ratio(df):
    sharpe = (df.mean()*252) / (df.std() * np.sqrt(252))
    return sharpe

In [ ]:
user_sharpe_ratio = sharpe_ratio(portfolio_df)
print(user_sharpe_ratio)

fake1   8.16
fake2   7.76
fake3   8.04
dtype: float64


In [ ]:
def return_on_investment(investment, returns):
    cumulative_profit = investment * returns
    return_oi = (cumulative_profit - investment) / investment
    chart_return_oi = return_oi.plot()
    plt.show()
    return(return_oi)

In [ ]:
user_roi = return_on_investment(initial_investment, user_cr)
user_roi